In [ ]:
# ------------------------
# define global settings
# ------------------------
SETTING_PIPELINE_NAME = "130_strava network experiment"

# ------------------------
# import packages
# ------------------------
import traceback
import requests
import time
import math
import os
import glob
import pprint
import json
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, f1_score
import pickle
import torch
import torch.nn as nn
from torch_geometric.sampler import BaseSampler
from torch_geometric.loader import NodeLoader,ImbalancedSampler,NeighborLoader, NodeLoader
from torch.utils.data import TensorDataset
import torch.optim as optim
from torch.autograd import Variable
# from torch_geometric_temporal.signal.static_graph_temporal_signal import StaticGraphTemporalSignal
# from torch_geometric_temporal.signal import temporal_signal_split
from torch.utils.data import SequentialSampler,DataLoader,WeightedRandomSampler

DIR_HOME = "E:\\Dropbox (MIT)\\202107_CovidExercise\\"
DIR_CURRENT = os.getcwd()
DIR_PIPELINE = os.path.join(DIR_HOME, "2_pipeline", SETTING_PIPELINE_NAME)
if not os.path.exists(DIR_PIPELINE):
    os.mkdir(DIR_PIPELINE)
    os.mkdir(os.path.join(DIR_PIPELINE, "out"))
    os.mkdir(os.path.join(DIR_PIPELINE, "store"))
    os.mkdir(os.path.join(DIR_PIPELINE, "temp"))

In [ ]:
# with open(os.path.join(DIR_PIPELINE, 'store\\USA MA Middlesex\\USA MA Middlesex data_F16_equal weight.pickle'), 'rb') as f:
with open(os.path.join(DIR_PIPELINE, 'store\\USA MA Middlesex\\USA MA Middlesex data.pickle'), 'rb') as f:
    temp_data = pickle.load(f)

In [ ]:
def train_scenario(scenario):
    s1 = [
        'user_gender', 'register_index',
        'TEMP', 'TEMP2', 'WIND', 'HUMI', 'HUMI2', 'VISI', 'PRES', 'CLOD', 'PRCP',
    ]
    s2 = [
        'user_gender', 'register_index',
        'TEMP', 'TEMP2', 'WIND', 'HUMI', 'HUMI2', 'VISI', 'PRES', 'CLOD', 'PRCP',
        'IS_COVID_START', 'POLICY',
    ]
    s3 = [
        'user_gender', 'register_index',
        'TEMP', 'TEMP2', 'WIND', 'HUMI', 'HUMI2', 'VISI', 'PRES', 'CLOD', 'PRCP',
        'IS_COVID_START', 'POLICY',
        'DURATION_1', 'DURATION_2', 'DURATION_3',
    ]
    s4 = [
        'user_gender', 'register_index',
        'TEMP', 'TEMP2', 'WIND', 'HUMI', 'HUMI2', 'VISI', 'PRES', 'CLOD', 'PRCP',
        'IS_COVID_START', 'POLICY',
        'LG_FOLLOW',
        'DURATION_1', 'DURATION_2', 'DURATION_3'
    ]
    if scenario == 1:
        return s1
    elif scenario == 2:
        return s2
    elif scenario == 3:
        return s3
    elif scenario == 4:
        return s4

def split_across_time(pdta, normal_test_point,covid_train_point,covid_test_point):
    train_dta = np.concatenate((pdta[:normal_test_point,::],pdta[covid_train_point:covid_test_point,::]))
    test_normal = pdta[normal_test_point:covid_train_point,::]
    test_covid = pdta[covid_test_point:,::]
    return train_dta, test_normal, test_covid

def splitting_gnn_data(gnn_dta,normal_test_point,covid_train_point,covid_test_point):
    dummy_y = temp_data['gnn_y'].copy()
    dummy_y[dummy_y>=THRESHOLD_150] = 1
    dummy_y[dummy_y<THRESHOLD_150] = 0
    dummy_y = np.array(dummy_y, dtype=np.long)

    edge_index = gnn_dta['edge_index']
    edge_attr = gnn_dta['edge_attr']
    gnn_X = gnn_dta['gnn_X']
    gnn_y = gnn_dta['gnn_y']


#     edge_index_train, edge_index_test_normal, edge_index_test_covid = split_across_time(edge_index,normal_test_point,covid_train_point,covid_test_point)
#     edge_attr_train, edge_attr_test_normal, edge_attr_test_covid = split_across_time(edge_attr,normal_test_point,covid_train_point,covid_test_point)
    gnn_X_train, gnn_X_test_normal, gnn_X_test_covid = split_across_time(gnn_X,normal_test_point,covid_train_point,covid_test_point)
    gnn_y_train, gnn_y_test_normal, gnn_y_test_covid = split_across_time(gnn_y,normal_test_point,covid_train_point,covid_test_point)
    dummy_train, dummy_test_normal, dummy_test_covid = split_across_time(dummy_y,normal_test_point,covid_train_point,covid_test_point)

    if FUNC_DUMMY_Y:
        gnn_train = StaticGraphTemporalSignal(
            temp_data['edge_index'],
            temp_data['edge_attr'],
            gnn_X_train,
            dummy_train)
        gnn_test_normal = StaticGraphTemporalSignal(
            temp_data['edge_index'],
            temp_data['edge_attr'],
            gnn_X_test_normal,
            dummy_test_normal)
        gnn_test_covid = StaticGraphTemporalSignal(
            temp_data['edge_index'],
            temp_data['edge_attr'],
            gnn_X_test_covid,
            dummy_test_covid)

    else:
        gnn_train = StaticGraphTemporalSignal(
            temp_data['edge_index'],
            temp_data['edge_attr'],
            gnn_X_train,
            gnn_y_train)
        gnn_test_normal = StaticGraphTemporalSignal(
            temp_data['edge_index'],
            temp_data['edge_attr'],
            gnn_X_test_normal,
            gnn_y_test_normal)
        gnn_test_covid = StaticGraphTemporalSignal(
            temp_data['edge_index'],
            temp_data['edge_attr'],
            gnn_X_test_covid,
            gnn_y_test_covid)

    return gnn_train, gnn_test_normal, gnn_test_covid

In [ ]:
FUNC_DUMMY_Y = False

normal_test_point = 44
covid_train_point = 54
covid_test_point = 118

THRESHOLD_150 = 0.28048000435422255
THRESHOLD_0 = -0.7569752353545586

# print(gnn_dataset[0])
gnn_train_dataset, gnn_test_normal_dataset, gnn_test_covid_dataset = splitting_gnn_data(temp_data,
                                                                                       normal_test_point,
                                                                                       covid_train_point,
                                                                                       covid_test_point)

In [ ]:
feature_map = {
    'user_gender': 'Gender (Female=1)',
    'register_index': 'Week index since registration',
    'TEMP': 'Temperature',
    'TEMP2': 'Temperature squared term',
    'WIND': 'Wind speed',
    'HUMI': 'Humidity',
    'HUMI2': 'Humidity squared term',
    'VISI': 'Visibility',
    'PRES': 'Pressure',
    'CLOD': 'Cloud coverage',
    'PRCP': 'Precipitation',
    'IS_COVID_START': 'Is COVID-19 started',
    'POLICY': 'Policy',
    'LG_FOLLOW': 'Log number of followings',
    'DURATION_1': '1 week lag of exercise duration',
    'DURATION_2': '2 week lag of exercise duration',
    'DURATION_3': '3 week lag of exercise duration'
}

In [ ]:
features = [feature_map[f] for f in features]

# Functions

In [ ]:
def scores(y_true, y_pred, mode):
    y_true = np.array(y_true, dtype=np.float64)
    y_pred = np.array(y_pred, dtype=np.float64)
    y_pred[y_pred<THRESHOLD_0] = THRESHOLD_0

    y_under_150 = (y_true<THRESHOLD_150) * 1
    y_above_150 = (y_true>=THRESHOLD_150) * 1

    pred_under_150 = (y_pred<THRESHOLD_150) * 1
    pred_above_150 = (y_pred>=THRESHOLD_150) * 1

    y_0_pred_0 = int(sum(y_under_150 & pred_under_150))
    y_0_pred_1 = int(sum(y_under_150 & pred_above_150))
    y_1_pred_0 = int(sum(y_above_150 & pred_under_150))
    y_1_pred_1 = int(sum(y_above_150 & pred_above_150))

    total = len(y_true)
    correct = int(y_0_pred_0) + int(y_1_pred_1)

    r2 = r2_score(y_true, y_pred)
    f1 = f1_score(y_above_150, pred_above_150)

    ret = {
        mode + ' y_0 pred_0': y_0_pred_0,
        mode + ' y_0 pred_1': y_0_pred_1,
        mode + ' y_1 pred_0': y_1_pred_0,
        mode + ' y_1 pred_1': y_1_pred_1,
        mode + ' lazy acc': y_0_pred_0 / (y_0_pred_0 + y_0_pred_1),
        mode + ' active acc': y_1_pred_1 / (y_1_pred_0 + y_1_pred_1),
        mode + ' r2_score': r2,
        mode + ' f1_score': f1,
        mode + ' mse': mean_squared_error(y_true, y_pred),
        mode + ' acc': correct/total
    }

    return ret


# Graph Neural Network

In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric_temporal.nn.recurrent import DCRNN, LRGCN, EvolveGCNO, GConvGRU

class RecurrentGCN(torch.nn.Module):
    def __init__(self, node_features):
        super(RecurrentGCN, self).__init__()
#         self.recurrent1 = GConvGRU(node_features, 16, 1)
        self.recurrent1 = DCRNN(node_features, 16, 1)
        self.ac1 = nn.SiLU()
        self.fc1 = nn.Linear(16, 16)
        self.ac2 = nn.SiLU()
#         self.recurrent2 = DCRNN(16, 16, 1)

        if TRAIN_MODE == 1:
            self.linear = torch.nn.Linear(16, 2)
        elif TRAIN_MODE == 2:
            self.linear = torch.nn.Linear(16, 1)

    def forward(self, x, edge_index, edge_weight):
        h = self.recurrent1(x, edge_index, edge_weight)
        h = self.ac1(h)
        h = self.fc1(h)
#         h = self.ac2(h)
#         h = self.recurrent2(h, edge_index, edge_weight)
        h = F.silu(h)
        h = self.linear(h)

#         if TRAIN_MODE == 1:
#             h = self.softmax(h)

        return h

## Train

In [ ]:
def gnn_evaluate(pmodel, data_loader, gpu=False, mode='unassigned'):
    y_true = np.array([])
    y_pred = np.array([])
    with torch.no_grad():
        for time, snapshot in enumerate(data_loader):
            slabels = snapshot.y.numpy()
            y_true = np.concatenate((y_true, slabels))

            y_hat = pmodel(snapshot.x, snapshot.edge_index, snapshot.edge_attr)

            if TRAIN_MODE == 1:
                predicted = torch.max(y_hat, 1).indices
            else:
                predicted = y_hat.reshape((-1,))

            spredicted = predicted.numpy()
            y_pred = np.concatenate((y_pred, spredicted))


    return scores(y_true, y_pred, mode), y_true, y_pred

def gnn_model_train(pmodel, train_data_loader, test_normal_data_loader,test_covid_data_loader, optimizer, gpu=False):
    best_accuracy = 0
    time = 0

    # Loss
    if TRAIN_MODE == 1:
        cri = torch.nn.CrossEntropyLoss(weight=torch.FloatTensor([0.7, 0.3]))
    elif TRAIN_MODE == 2:
        cri = torch.nn.MSELoss()

    ret = []
    for epoch in tqdm(list(range(151))):
        cost = 0
        for time, snapshot in enumerate(train_data_loader):
            y_hat = pmodel(snapshot.x, snapshot.edge_index, snapshot.edge_attr)

            if TRAIN_MODE == 1:
#                 y_pred = torch.max(y_hat, 1).indices
#                 print(y_hat)
#                 print(y_pred)
#                 print(snapshot.y)
#                 raise Exception("Error")
                cost = cost + cri(y_hat, snapshot.y)
            elif TRAIN_MODE == 2:
                y_hat = y_hat.reshape((-1,))
                cost = cost + torch.mean((y_hat-snapshot.y)**2)
        cost = cost / (time+1)


        cost.backward()
#         print(cost)
        optimizer.step()
        optimizer.zero_grad()

#         print(snapshot.y)
#         print(y_hat)

        if epoch % 10 == 0:
            train_ret,_,_ = gnn_evaluate(pmodel, train_data_loader, gpu=gpu, mode='train')
            test_normal_ret,_,_ = gnn_evaluate(pmodel, test_normal_data_loader, gpu=gpu, mode='test normal')
            test_covid_ret,_,_ = gnn_evaluate(pmodel, test_covid_data_loader, gpu=gpu, mode='test covid')
            print(f"epoch-{epoch}, train_mse: {train_ret['train mse']:3f}, train_acc: {train_ret['train acc']:3f}")
            print(f"               test_normal_mse: {test_normal_ret['test normal mse']:3f}, test_normal_acc: {test_normal_ret['test normal acc']:3f}")
            print(f"               test_covid_mse: {test_covid_ret['test covid mse']:3f}, test_covid_acc: {test_covid_ret['test covid acc']:3f}")
            temp_ret = {'epoch': epoch}
            temp_ret.update(train_ret)
            temp_ret.update(test_normal_ret)
            temp_ret.update(test_covid_ret)
            ret.append(train_ret)
            ret.append(test_normal_ret)
            ret.append(test_covid_ret)

            agg_acc = (test_normal_ret['test normal acc'] + test_covid_ret['test covid acc']) / 2
            if agg_acc >= best_accuracy:
                best_accuracy = agg_acc
                torch.save(pmodel, os.path.join(DIR_PIPELINE, 'store/model/gnn_unweight_twolayer DCRNN.pth'))

    return pmodel, ret

In [ ]:
THRESHOLD_150 = 0.28048000435422255
THRESHOLD_0 = -0.7569752353545586
# TRAIN_MODE == 1: classification
# TRAIN_MODE == 2: regression
TRAIN_MODE = 2
gnn_model = RecurrentGCN(node_features = 16)

In [ ]:
optimizer = torch.optim.Adam(gnn_model.parameters(), lr=0.001)
gnn_model.train()
gnn_model, gnn_ret = gnn_model_train(gnn_model, gnn_train_dataset, gnn_test_normal_dataset,gnn_test_covid_dataset, optimizer, True)

## Evaluate

In [ ]:
gnn_model = torch.load(os.path.join(DIR_PIPELINE, 'store/model/gnn_unweight_twolayer DCRNN.pth'))

In [ ]:
gnn_model.eval()
ret1,_,_ = gnn_evaluate(gnn_model, gnn_train_dataset, mode='train')
ret2,_,_ = gnn_evaluate(gnn_model, gnn_test_normal_dataset, mode='test normal')
ret3,_,_ = gnn_evaluate(gnn_model, gnn_test_covid_dataset, mode='test covid')
pprint.pprint(ret1)
pprint.pprint(ret2)
pprint.pprint(ret3)
